<a href="https://colab.research.google.com/github/mukthakaja/microsoft-phi-3-slm-support-agent/blob/main/phi3_fine_tune_qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq --upgrade bitsandbytes transformers peft accelerate datasets
!pip install huggingface_hub
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which 

In [ ]:
!pip install wandb
!pip install trl
!pip install absl-py nltk rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fd95ad3c5493cb0c839a632b8af710ab985bcde0ea98af8d55c3542a81c47660
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from random import randrange
import torch
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, peft_model

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed, pipeline
from trl import SFTTrainer


In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"
model_name = "microsoft/Phi-3-mini-4k-instruct"

dataset_name = "rghosh8/supportGPT_data"
dataset_split = "train"
new_model = "fine_tuned_qlora_phi3"

device_map = {"": 0}
use_4bit = True
bnb_4bit_compute_dtype = "bfloat16"
bnb_4bit_quant_type = "nf4"
use_double_quant = True

lora_r = 16
lora_alpha = 16
lora_dropout = 0.05
target_modules = "all-linear"

set_seed(47)

In [ ]:
dataset = load_dataset(dataset_name, split = dataset_split)
print (dataset)

Generating train split:   0%|          | 0/149 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 149
})


In [ ]:
tokenizer_id = model_id
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import AutoTokenizer
from datasets import DatasetDict

split_dataset = dataset.train_test_split(test_size = 0.2)
split_dataset = DatasetDict({
    'train' : split_dataset['train'],
    'validation': split_dataset['test']
})

tokenizer = AutoTokenizer.from_pretrained(model_name)

def format(samples) :
  return tokenizer(samples['text'], padding = "max_length", truncation = True)

tokenized_datasets = split_dataset.map(format, batched = True)

print(tokenized_datasets['train'][2])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

{'text': '### Human: Can you tell about Genesis crashing "No vmknic exists in same subnet as eth0/eth2". ### Assistant: This article describes an issue where genesis is crashing with the message "No vmknic exists in same subnet as eth0/eth2.". While adding a new node to a cluster, the node is not being discovered because genesis is crashing and Foundation is not starting.\n\ngenesis.out log shows:\n\nnutanix@CVM~$ less ~/data/logs/genesis.out. There are two possible reasons for the issue:\n\nThe CVM eth0 has an APIPA address.&nbsp;Modify /etc/sysconfig/network-scripts/ifcfg-eth0 and assign a static IP address, gateway and network mask. Change BOOTPROTO="dhcp" to BOOTPROTO="none".\n\n\tHere is an example of an ifcfg-eth0 output:\n\n\tnutanix@CVM~$ vi /etc/sysconfig/network-scripts/ifcfg-eth0\n\tHost vmk port has APIPA address assigned.&nbsp;The Host vmknic IP configuration can be checked with esxcfg-vmknic -l. Assign a static IP either through IPMI/DCUI or if there is an ssh connection 

In [ ]:
if torch.cuda.is_bf16_supported() :
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else :
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

print(attn_implementation)
print(compute_dtype)

flash_attention_2
torch.bfloat16


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, add_eos_token = True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'left'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_double_quant,
)

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = compute_dtype, trust_remote_code=True, quantization_config = bnb_config, attn_implementation = attn_implementation)

model = prepare_model_for_kbit_training(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import get_peft_model, TaskType
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
    task_type = TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)

In [ ]:
training_args = TrainingArguments(
   evaluation_strategy = "steps",
   output_dir = new_model,
   per_device_train_batch_size = 1,
   per_device_eval_batch_size = 1,
   learning_rate = 2e-5,
   weight_decay = 0.001,
   fp16 = False,
   max_steps = 1000,
   num_train_epochs = 1,
   save_strategy = "epoch",
   gradient_accumulation_steps = 16,
   gradient_checkpointing = True,
   logging_steps = 10,
   optim = "paged_adamw_32bit",
   lr_scheduler_type="linear"

)


trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    peft_config = peft_config,
    dataset_text_field = "text",
    max_seq_length = 512,
    tokenizer = tokenizer,
    args = training_args
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will over

In [ ]:
!pip uninstall wandb

Found existing installation: wandb 0.17.5
Uninstalling wandb-0.17.5:
  Would remove:
    /usr/local/bin/wandb
    /usr/local/bin/wb
    /usr/local/lib/python3.10/dist-packages/package_readme.md
    /usr/local/lib/python3.10/dist-packages/wandb-0.17.5.dist-info/*
    /usr/local/lib/python3.10/dist-packages/wandb/*
Proceed (Y/n)? Y
  Successfully uninstalled wandb-0.17.5


In [ ]:
!pip install wandb

In [ ]:
!pip install flash-attn

In [ ]:
!pip install -q -U peft transformers datasets bitsandbytes trl accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
!pip install huggingface_hub
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

model_id = "microsoft/Phi-3-mini-4k-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type = "nf4"
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)
tokenizer.add_special_tokens({'pad_token' : '<PAD'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

1

In [ ]:
peft_config = LoraConfig(
    r = 16,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias = "none",
    task_type = "CAUSAL_LM"
)

model.add_adapter(peft_config)

In [ ]:
from datasets import load_dataset

dataset_name = "rghosh8/supportGPT_data"

split_dataset = load_dataset(dataset_name, split="train[:1%]")

Generating train split:   0%|          | 0/149 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

hf_username = "mukthakaja"

training_arguments = TrainingArguments (
    output_dir = f"{hf_username}/finetuned_phi3_qlora",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    learning_rate = 2e-5,
    max_steps = 200,
    num_train_epochs = 1,
    save_strategy = "epoch",
    gradient_accumulation_steps = 16,
    gradient_checkpointing = True,
    logging_steps = 10,
    optim = "paged_adamw_32bit",
    lr_scheduler_type="linear"
)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from trl import SFTTrainer

def format(samples) :
  return tokenizer(samples['text'], padding = "max_length", truncation = True)

trainer = SFTTrainer(
    model = model,
    args = training_arguments,
    train_dataset = split_dataset,
    dataset_text_field = "text",
    tokenizer = tokenizer,
    max_seq_length = 1024,
    formatting_func = format,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


Step,Training Loss
10,0.113000
20,0.112300
30,0.111200
40,0.109900
50,0.108400
60,0.106900
70,0.105200
80,0.103600
90,0.101900
100,0.100300


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed 

TrainOutput(global_step=200, training_loss=0.10074471801519394, metrics={'train_runtime': 659.2507, 'train_samples_per_second': 4.854, 'train_steps_per_second': 0.303, 'total_flos': 4585257172992000.0, 'train_loss': 0.10074471801519394, 'epoch': 200.0})

In [ ]:
finetuned_model = "https://huggingface.co/mukthakaja/finetuned_phi3_qlora"

trainer.model.save_pretrained(finetuned_model)
tokenizer.save_pretrained(finetuned_model)

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


('mukthakaja/finetuned_phi3_qlora/tokenizer_config.json',
 'mukthakaja/finetuned_phi3_qlora/special_tokens_map.json',
 'mukthakaja/finetuned_phi3_qlora/tokenizer.model',
 'mukthakaja/finetuned_phi3_qlora/added_tokens.json',
 'mukthakaja/finetuned_phi3_qlora/tokenizer.json')

In [ ]:
trainer.model.save_pretrained(f"{hf_username}/finetuned_phi3_qlora")

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

hf_username = "mukthakaja"

finetuned_model_id = f"{hf_username}/finetuned_phi3_qlora"

tokenizer = AutoTokenizer.from_pretrained(finetuned_model_id)


model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_id
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text = "### Human: Can you tell about All user VMs will not be powered on automatically after power outage in a 2 node Nutanix AHV cluster. ### Assistant:"

inputs = tokenizer(text, return_tensors = "pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens = 250, do_sample = False)

print(tokenizer.decode(outputs[0], skip_special_tokens = False))

### Human: Can you tell about All user VMs will not be powered on automatically after power outage in a 2 node Nutanix AHV cluster. ### Assistant: 
Certain explanation:

In a Nutanix AHV (Acropolis Hyper-Converged Infrastructure) cluster, VMs (Virtual Machines) are managed by the Nutanix AHV Manager. By default, VMs are not powered on automatically after a power outage. This is a safety measure to prevent accidental data loss or corruption that could occur if a power outage happens while a VM is in the middle of a critical operation.

However, you can configure your AHV cluster to power on VMs automatically after a power outage. This can be done by setting the "Power On After Outage" option in the AHV Manager.

Here are the steps to configure this:

1. Open the Nutanix AHV Manager.
2. Navigate to the "Clusters" tab and select the cluster you want to configure.
3. Click on the "Power Management" tab.
4. Check the "Power On After Outage" option.
5. Click "Apply" to save the changes.

Rem